In [7]:
# !pip install -r ../exp_requirements.txt

In [3]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [6]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

# Credit Dataset With Null Imputer

## Import dependencies

In [7]:
import os
import copy

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from virny.utils.custom_initializers import create_config_obj
from virny.datasets.data_loaders import CreditDataset

from source.preprocessing import get_null_imputer_preprocessor
from source.experiment_interface import run_exp_iteration
from configs.constants import NUM_METRICS_COMPUTATION_RUNS, EXPERIMENT_SEEDS

## Initialize input variables for the experiment

In [ ]:
ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
EXPERIMENT_NAME = 'preprocessing'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)

In [ ]:
data_loader = CreditDataset(subsample_size=50_000)
data_loader.X_data.head()

In [ ]:
data_loader.full_df.isna().sum()

### Define a db writer and custom fields to insert into your database

In [ ]:
import uuid

custom_table_fields_dct = {
    'notebook_session_uuid': str(uuid.uuid4()),
    'preprocessing_technique': None,
}
print('Current session uuid: ', custom_table_fields_dct['notebook_session_uuid'])

### Create a metrics computation config object

In [ ]:
config_yaml_path = 'experiment_config.yaml'
age_range = [i for i in range(0, 31)]
config_yaml_content = \
f"""
dataset_name: Credit
bootstrap_fraction: 0.8
n_estimators: 50  # Better to input the higher number of estimators than 100; this is only for this use case example
num_runs: {NUM_METRICS_COMPUTATION_RUNS}
sensitive_attributes_dct: {{'age': {age_range}}}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [ ]:
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

### Define tuning parameter for models

In [ ]:
def get_models_params_for_tuning(models_tuning_seed):
    return {
        'DecisionTreeClassifier': {
            'model': DecisionTreeClassifier(random_state=models_tuning_seed),
            'params': {
                "max_depth": [20, 30],
                "min_samples_split" : [0.1],
                "max_features": ['sqrt'],
                "criterion": ["gini", "entropy"]
            }
        },
        'LogisticRegression': {
            'model': LogisticRegression(random_state=models_tuning_seed),
            'params': {
                'penalty': ['l2'],
                'C' : [0.0001, 0.1, 1, 100],
                'solver': ['newton-cg', 'lbfgs'],
                'max_iter': [250],
            }
        },
        'RandomForestClassifier': {
            'model': RandomForestClassifier(random_state=models_tuning_seed),
            'params': {
                "max_depth": [6, 10],
                "min_samples_leaf": [1],
                "n_estimators": [50, 100],
                "max_features": [0.6]
            }
        },
        'XGBClassifier': {
            'model': XGBClassifier(random_state=models_tuning_seed, verbosity=0),
            'params': {
                'learning_rate': [0.1],
                'n_estimators': [200],
                'max_depth': [5, 7],
                'lambda':  [10, 100]
            }
        },
        # 'KNeighborsClassifier': {
        #     'model': KNeighborsClassifier(),
        #     'params': {
        #         'n_neighbors' : [5, 7, 9, 11, 13, 15, 25],
        #         'weights' : ['uniform', 'distance'],
        #         'metric' : ['minkowski', 'euclidean', 'manhattan']
        #     }
        # },
        # 'MLPClassifier': {
        #     'model': MLPClassifier(random_state=models_tuning_seed),
        #     'params': {
        #         'hidden_layer_sizes':[(100,), (100,100,), (100,50,100,)],
        #         'activation': ['logistic', 'tanh', 'relu'],
        #         'solver': ['lbfgs', 'sgd', 'adam'],
        #         'learning_rate': ['constant', 'invscaling', 'adaptive']
        #     }
        # }
    }

## Run experiments

### Experiment iteration 1

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['preprocessing_technique'] = 'cat: mode, num: median'
custom_table_fields_dct['experiment_iteration'] = exp_iter_num
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)
models_params_for_tuning = get_models_params_for_tuning(experiment_seed)

In [ ]:
multiple_run_metrics_dct = run_exp_iteration(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             preprocessor=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             with_tuning=True,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             tuned_params_df_path=None)

In [ ]:
sample_model_metrics_df = multiple_run_metrics_dct[list(models_params_for_tuning.keys())[0]]
sample_model_metrics_df.head(20)

### Experiment iteration 2